In [1]:
pip install torch sklearn transformers tensorflow datasets spacy

Looking in indexes: https://nexus.irn.intel.politie/repository/pypi-politie/simple
Note: you may need to restart the kernel to use updated packages.


In [17]:
import spacy
import torch
import random
import json
import re
import tensorflow as tf
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import datasets
from datasets import load_dataset, load_metric
import string
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    BertModel,
    BertTokenizer,
    DataCollatorForLanguageModeling, 
    BertForPreTraining,
    BertConfig,
    BertForMaskedLM,
    Trainer,
    TrainingArguments
)
from transformers.trainer_utils import get_last_checkpoint

In [6]:
#pip install 'en_core_web_lg.tar.gz'
nlp = spacy.load('en_core_web_lg-3.2.0/en_core_web_lg/en_core_web_lg-3.2.0')

In [31]:
#load in pre-trained BERT model
model = BertForPreTraining.from_pretrained('model')
tokenizer = BertTokenizer.from_pretrained('model')

loading configuration file model/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

loading weights file model/pytorch_model.bin
All model checkpoint weights were used when initializing BertForPreTraining.

Some weights of BertForPreTraining were not initialize

In [48]:
#load in dataset, delete unnecesary columns & split train and test

dataset = load_dataset('csv', data_files = "IMDB-review-sentiment.csv")
dataset.keys()
dataset = dataset.map(lambda x: {"len":len(x['review'])})
dataset = dataset.map(lambda x: {"next_sentence_label":0})
max(dataset['train']['len'])
dataset = dataset['train'].train_test_split(test_size=0.2)
    
#truncate the reviews to maximum 128 characters
preprocessed_dataset = dataset.map(lambda x: {"input_ids":tokenizer(x['review'], truncation=True, max_length=128)["input_ids"]})
    
#declare the data collator: takes as input the dataset and collates it into a dictionary of Tensors. 
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)
   
#more tokens are added to the existing tokenizer, so we need to resize the model's token embeddings in order to fit.
model.resize_token_embeddings(len(tokenizer))

Using custom data configuration default-0cd2026000bdb6b0
Reusing dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-0cd2026000bdb6b0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 1/1 [00:00<00:00, 214.30it/s]
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-0cd2026000bdb6b0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-ac7e3b60ed8a41f4.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-0cd2026000bdb6b0/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-bb5c1fcb8dbfcd52.arrow
100%|██████████| 10000/10000 [01:18<00:00, 126.61ex/s]


Embedding(119547, 768, padding_idx=0)

In [32]:
#instantiating training arguments
output_dir = "./model/output/test2"
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    prediction_loss_only =True,
    do_train=True,
    per_device_train_batch_size=32,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [33]:
#declare model, dataset and trainingargs etc.
trainer = Trainer(
    model = model, 
    args = training_args, 
    data_collator = data_collator, 
    train_dataset = preprocessed_dataset['train'],
    eval_dataset = preprocessed_dataset['test'],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForPreTraining.forward` and have been ignored: sentiment, review, len. If sentiment, review, len are not expected by `BertForPreTraining.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Step,Training Loss


In [35]:
model.save_pretrained('./model/output_final/bert2')

Configuration saved in ./model/output_final/bert1/config.json
Model weights saved in ./model/output_final/bert1/pytorch_model.bin
